#FastAPI 
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

The key features are:

* Fast: Very high performance, on par with NodeJS and Go (thanks to Starlette and Pydantic). One of the fastest Python frameworks available.
* Fast to code: Increase the speed to develop features by about 200% to 300%. *
* Fewer bugs: Reduce about 40% of human (developer) induced errors. *
* Intuitive: Great editor support. Completion everywhere. Less time debugging.
* Easy: Designed to be easy to use and learn. Less time reading docs.
* Short: Minimize code duplication. Multiple features from each parameter declaration. Fewer bugs.
* Robust: Get production-ready code. With automatic interactive documentation.
* Standards-based: Based on (and fully compatible with) the open standards for APIs: OpenAPI (previously known as Swagger) and JSON Schema.

## Asyncio
Antes de empezar con FastAPI hay que explicar como funcionan programas asincronicos.

```python
import asyncio

async def fetch_data():
  print('start fetching')
  await asyncio.sleep(1)
  print('done_fetching')
  return {"data": 1}

async def print_numbers():
  for i in range(10):
    print(i)
    await asyncio.sleep(0.25)

async def main():
  task1 = asyncio.create_task(fetch_data())
  task1 = asyncio.create_task(print_numbers())

  value = await task1
  print(value)

asyncio.run(main())
```

Aqui definimos dos funciones asincronicas con `async`, y creamos tasks con estas funciones. 
Si no aplicamos `await` con los tasks, nomas va a terminar el programa mientras los tasks corren en el fondo.
Si nada mas aplicamos `await` con el primer task, va a terminar mientras corra el segundo, y ya no vamos a ver como termina el segundo task. Para ver todos los prints, tenemos que esperar a los dos tasks.
Si nada mas aplicamos `await` con el segundo task, no vamos a ser capaces de imprimir/leer lo que retorna el primer task.

Lo que pasa aqui es que siempre se ejecuta nada mas un task, pero cuando estan en espera, podemos avanzar con el otro task.

Para nuestra API podria significar que podemos procesar un request, en el que requerimos datos de una base de datos externa, mientras estamos esperando a la base de datos, podemos saltar a otro request y ya empezar a procesarlo. 

Correr procesos asincronicos es diferente a correr procesos en paralelo.

Correr procesos sincronicos significa procesarlo de manera secuencial, mientras en procesos asincronicos podemos saltar de un proceso a otro.

En este video, pueden encontrar una explicacion mucho mas a detalle, de lo que es correr procesos asincronicos.

https://www.youtube.com/watch?v=t5Bo1Je9EmE

## Basic FastAPI without Pydantic





### Installation

La forma preferida de la instalacion de FastAPI requiere usar pip. Tambien necesitamos instalar uvicorn para poder montar el servidor web.

```
pip install fastapi
pip install "uvicorn[standard]"
```



### First steps

Para poder armar un ejemplo basico de FastAPI, lo unico que necesitamos son los imports, luego crear la instancia de FastAPI y finalmente definir la ruta de API con una funcion.
La ruta de la API se crea a traves de un `path operation decorator`. La operacion se define como funcion de la instancia `app`, `app.get()`. Dentro de aquella funcion definimos el path, en este caso `"/"`, `app.get("/")`.
La `path operation function` define como procesar los datos de entrada, y como regresarlos. Podemos definir la funcion como asincronica con el keyword `async` o sincronica.

main.py
```python
from fastapi import FastAPI
import names

app = FastAPI()

# Path operation decorator
@app.get("/")
# Path operation function
async def root():
    return {"message": "Hello World"}

@app.get("/random_name")
async def root():
    return {"message": f"Hello {names.get_first_name()}"}

```

Esto es todo lo que necesitamos hacer, ahora nomas nos falta iniciar el servidor web con uvicorn, apuntando a nuestro archivo `main.py` o cualquier otro deseado, y a la instancia `app`. El keyword `reload` permite ver cambios a la API de forma instantanea, sin tener que volver a inicia el servidor web.
Start server:
```
uvicorn main:app --reload
```

Ahora podemos explorar nuestra Api con los siguientes enlaces:
https://127.0.0.1:8000

http://127.0.0.1:8000/docs

http://127.0.0.1:8000/redoc

### Path Parameters

Una funcionalidad muy importante de API's en general son los `Path Parameters`.
Podemos definir algunas partes nuestro path como variables, lo cual nos permite usar los parametros en nuestra funcion como una variable normal de entrada. En este ejemplo lo unico que hacemos es regresar la variable que nos pasa el usuario en la respuesta.

Checa como funciona en  https://127.0.0.1:8000/songs/4
```python
from fastapi import FastAPI

app = FastAPI()


song_db = ["evermore", "willow", "champagne problems", "gold rush",
           "'tis the damn season'", "tolerate it", "no body no crime",
           "happiness", "dorothea", "coney island",
           "ivy", "cowboy like me", "long story short"]


@app.get("/name/{name}")
async def get_name(name):
    return {"message": f"Hello {name}"}


@app.get("/songs/{song_id}")
async def get_song(song_id):
    return {"song_id": song_id, "song_name": song_db[song_id]}

    
```




### Path parameters with types

A veces queremos limitar nuestros parametros a un cierto tipo de datos, FastAPI nos permite a hacer esto de manera muy simple (con la ayuda de Pydantic) a traves del `typing`. Podemos definir el tipo de una variable y a partir de ahora, nuestra API no va a permitir otro tipo cuando la llamamos.

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/name/{name}")
async def get_name(name: str):
    return {"message": f"Hello {name}"}


song_db = ["evermore", "willow", "champagne problems", "gold rush",
           "'tis the damn season'", "tolerate it", "no body no crime",
           "happiness", "dorothea", "coney island",
           "ivy", "cowboy like me", "long story short"]


@app.get("/songs/{song_id}")
async def get_song(song_id: int):
    return {"song_id": song_id, "song_name": song_db[song_id]}
```
Funciona:
http://127.0.0.1:8000/songs/3

http://127.0.0.1:8000/name/daniel

No funciona:
http://127.0.0.1:8000/songs/daniel

Tambien funciona:
http://127.0.0.1:8000/name/3

En este caso, FastAPI hace parsing automatico del numero y lo convierte a un string. Podemos forzar FastAPI a rechazar numericos con `StrictStr``
```python
from pydantic import StrictStr
@app.get("/name/{name}")
async def get_name(name: StrictStr):
    return {"message": f"Hello {name}"}

```


Para ajustar nuestros parametros podemos usar un Path object. Con este Path object le podemos dar por ejemplo un titulo, o definir un rango para los tipos numericos.
Porque los parametros Path siempre son requeridos, porque forman parte del path, no es posible darle un valor default.

```python
from fastapi import Path

@app.get("/songs/{song_id}")
async def get_song(song_id: int = Path(
  ..., title="The ID of the item to get", le=12)):
    return {"song_id": song_id, "song_name": song_db[song_id]}

    
```
Tambien FastAPI ya nos regresa informacion por que no funciona el ejemplo, y tambien menciona este requerimiento en la documentacion de nuestra API:
http://127.0.0.1:8000/docs



### Order matters

Cuando creamos rutas importa mucho el orden de ejecuccion, en este ejemplo tenemos una ruta hardcoded (evermore) y despues la ruta con el parametro. Si ahora llamamos http://127.0.0.1:8000/songs/evermore nos va a regresar esta cancion. Pero si le damos vuelta a la orden, nos fallaria la misma llamada, porque la ruta requiere un int y evermore no es int.

```python
from fastapi import FastAPI

app = FastAPI()

song_db = ["evermore", "willow", "champagne problems", "gold rush",
           "'tis the damn season'", "tolerate it", "no body no crime",
           "happiness", "dorothea", "coney island",
           "ivy", "cowboy like me", "long story short"]


@app.get("/songs/evermore")
async def get_evermore():
    return {"song_id": "0",
            "song_name": "evermore"}


@app.get("/songs/{song_id}")
async def get_song(song_id: int):
    return {"song_id": song_id, "song_name": song_db[song_id]}

```



### Predefined values

Podemos usar enums para predefinir los valores que aceptamos en una ruta. Ahora nuestra ruta nada mas acepta los valores que definimos en el Enum, y en otros casos nos tira un error. Tambien lo vamos a poder observar en la documentacion.

Funciona: http://127.0.0.1:8000/albums/evermore

No funciona: http://127.0.0.1:8000/albums/anything

http://127.0.0.1:8000/docs


```python
from enum import Enum

from fastapi import FastAPI


class AlbumName(str, Enum):
    folklore = "folklore"
    evermore = "evermore"
    red = "red"


app = FastAPI()


@app.get("/album/{album_name}")
async def get_album(album_name: AlbumName):
    if album_name == AlbumName.folklore:
        return {"album_name": album_name, "year": 2020}
    if album_name.value == "evermore":
        return {"album_name": album_name, "year": 2020}
    return {"album_name": album_name, "year": 2021}
```



### Query Parameters

Otro tipo de parametro se llama `query parameters`. Los podemos definir al final de una query con `?`, por ejemplo http://127.0.0.1:8000/items/?skip=0.
Para pasar varios query parameters podemos concatenarlos con `&`, e.g.
http://127.0.0.1:8000/items/?skip=0&limit=10

```python

from fastapi import FastAPI

app = FastAPI()

song_db = ["evermore", "willow", "champagne problems", "gold rush",
           "'tis the damn season'", "tolerate it", "no body no crime",
           "happiness", "dorothea", "coney island",
           "ivy", "cowboy like me", "long story short"]

@app.get("/song_list")
async def get_song_list(skip: int = 0, limit: int = 10):
    return song_db[skip: skip + limit]
````

Parecido al Path Object podemos usar un Query object para definir mas opciones para los query parameters. Los Query Objects pueden ser opcionales, para hacerlos obligatorio, no hay que pasar un valor default, or usar un ellipsis (...). En el ejemplo de abajo usamos un elllipsis, haciendo el primer parametro obligatorio.

```python
@app.get("/song_list")
async def get_song_list(skip: int = Query(..., le=12), limit: int = Query(10, le=12)):
    return song_db[skip: skip + limit]
```

## Pydantic

Pydantic es un modulo de Python que nos ayuda a hacer parsing y validacion de datos. Funciona muy bien con JSON, lo que luego vamos a usar para mandar datos mas extensivos a nuestra API.
Con Pydantic podemos definir clases, que evaluan JSON o otros formatos y los guarda en un objetivo nativo a Python.
### Installation

Pero primero hay que instalar `pydantic`


In [2]:
%pip install pydantic

     |████████████████████████████████| 10.9 MB 5.1 MB/s 


### Basic Pydantic
Con Pydantic podemos crear clases, que defien varios campos. Si no pasamos un valor default, el campo es obligatorio. Podemos crear una instancia de la clase con JSON/dicts o tambien con los campos o una combinacion.

In [11]:
from typing import List, Optional
from pydantic import BaseModel
from datetime import datetime

class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)
print(user.id)
print(user)
external_data.pop("friends")
user = User(**external_data, name="Mike Pence", friends=[])
print(user)


123
id=123 signup_ts=datetime.datetime(2019, 6, 1, 12, 22) friends=[1, 2, 3] name='John Doe'
id=123 signup_ts=datetime.datetime(2019, 6, 1, 12, 22) friends=[] name='Mike Pence'


### Constraints and validators
Tambien podemos usar constraints y validators para definir bien nuestros requerimientos para los campos.

In [ ]:
from pydantic import (
    BaseModel,
    NegativeInt,
    PositiveInt,
    conint,
    conlist,
    constr
)
class Model(BaseModel):
    # minimum length of 2 and maximum length of 10
    short_str: constr(min_length=2, max_length=10)
    # regex
    regex_str: constr(regex=r'^apple (pie|tart|sandwich)$')
    # remove whitespace from string
    strip_str: constr(strip_whitespace=True)

    # value must be greater than 1000 and less than 1024
    big_int: conint(gt=1000, lt=1024)
    
    # value is multiple of 5
    mod_int: conint(multiple_of=5)
    
    # must be a positive integer
    pos_int: PositiveInt
    
    # must be a negative integer
    neg_int: NegativeInt

    # list of integers that contains 1 to 4 items
    short_list: conlist(int, min_items=1, max_items=4)

In [65]:
from pydantic import validator
from typing import List
class User(BaseModel):
    id: conint(ge=10)
    name: constr(min_length=5, strip_whitespace=True) = 'John Doe'
    signup_ts: datetime = None
    friends: List[int] = []

    @validator('id')
    def id_must_be_4_digits(cls, v):
        if len(str(v)) != 4:
            raise ValueError('must be 4 digits')
        return v

external_data = {
  
    'id': '1234',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
    'name': 'Jordan Davis'
  

}


id=1234 name='Andre Bajorat' signup_ts=datetime.datetime(2019, 6, 1, 12, 22) friends=[1, 2, 3]


### Nested models
Tambien podemos usar nested models, es decir definir una clase y usarla en otra clase.

In [72]:
class UserList(BaseModel):
    users: List[User]
    description: str = "All the users"

external_data = {
    'users':[
    {
      'id': '1234',
      'signup_ts': '2019-06-01 12:22',
      'friends': [1, 2, '3'],
      'name': ' Jordan Davis '
    },
    {
      'id': '1234',
      'signup_ts': '2019-06-01 12:22',
      'friends': [1, 2, '3'],
      'name': 'Jimmy G'
    }
  ]
}
users = UserList(**external_data)
print(users.users)

        

[User(id=1234, name='Jordan Davis', signup_ts=datetime.datetime(2019, 6, 1, 12, 22), friends=[1, 2, 3]), User(id=1234, name='Jimmy G', signup_ts=datetime.datetime(2019, 6, 1, 12, 22), friends=[1, 2, 3])]


## More FastAPI

### Request Body
Podemos usar pydantic para crear clases que definen el input que esperamos en una llamada API. Cuando queremos mandar mas datos, ya no es buena idea usar path y query parametros. En vez de hacer esto, podemos mandar un JSON en el body del request.
Este body puedeser parseado conforme con una clase de Pydantic. Para hacer esto, nada mas hay que definir la clase y usar aquella clase como tipo del parametro para la path operation function.


```python

from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel
app = FastAPI()


class Album(BaseModel):
    name: str
    description: str = None   
    price: confloat(gt=0) = 'No description given'
    discount:float = None



@app.post("/albums/")
async def create_album(album: Album):
    album_dict = album.dict()
    ## save album
    return album_dict
```

Tambien podemos transformar los datos antes de regresarlos.

```python
@app.post("/albums/")
async def create_album(album: Album):
    album_dict = album.dict()
    new_price = (album.price - album.discount * album.price * 0.01)
    album_dict.update({"new_price": new_price})
    return album_dict
```



### Response Models

No solo podemos parsear los datos de entrada, sino tambien los datos que mandamos de regreso. Para esto se usa el parametro `response_model` en el decorator.

```python
from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Album(BaseModel):
    name: str
    description: str = None   
    price: confloat(gt=0) = 'No description given'
    discount:float = 0

class AlbumOut(BaseModel):
    name: str
    description: str = None  
    new_price: float = None 

@app.post("/albums/", response_model=AlbumOut)
async def create_album(album: Album):
    album_dict = album.dict()
    new_price = (album.price - album.discount * album.price * 0.01)
    album_dict.update({"new_price": new_price})
    return album_dict
```



### Authentication
No deberiamos exponer nuestra API publicamente. Obviamente la seguridad de API's es un tema que podria llenar muchas sesiones, por lo cual nada mas vamos a ver lo mas basico. Este ejemplo no se puede considerar una opcion muy segura, pero es mejor que nada.

Lo que hacemos aqui es usar un authorization token en el header, el cual verificamos en la funcion de verify_token. El token es estatico, unhashed y se usa uno solo para todos los usuarios. Cada una de estas caracteristicas es suficiente para considerar la seguridad de nuestra API insuficiente.

En el header de nuestros requests tenemos que definir lo siguiente para poder acceder a nuestra API.

`"Authorization": "BestPossiblePassword"`

```python
from fastapi import FastAPI, HTTPException, Depends, Request


@app.get("/check", dependencies=[Depends(verify_token)])
async def home():
    return {"detail": "Welcome home"}

def verify_token(req: Request):
    token = req.headers.get("Authorization", None)
    valid = token == 'BestPossiblePassword'
    if not valid:
        raise HTTPException(
            status_code=401,
            detail="Unauthorized"
        )
    return True

```


https://towardsdatascience.com/how-to-deploy-a-machine-learning-model-with-fastapi-docker-and-github-actions-13374cbd638a
https://kinsta.com/blog/generate-ssh-key/

https://fastapi.tiangolo.com/tutorial/

https://betterprogramming.pub/the-beginners-guide-to-pydantic-ba33b26cde89

https://www.youtube.com/watch?v=t5Bo1Je9EmE